In [1]:
!rm -r ../.local/share/Trash/files

rm: cannot remove '../.local/share/Trash/files': No such file or directory


In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          48G   29G   17G  64% /
tmpfs            64M     0   64M   0% /dev
tmpfs            45G     0   45G   0% /sys/fs/cgroup
shm             1.0G     0  1.0G   0% /dev/shm
/dev/xvda1       48G   29G   17G  64% /etc/hosts
tmpfs            45G   12K   45G   1% /proc/driver/nvidia
udev             45G     0   45G   0% /dev/nvidia0
tmpfs            45G     0   45G   0% /proc/acpi
tmpfs            45G     0   45G   0% /proc/scsi
tmpfs            45G     0   45G   0% /sys/firmware


In [3]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco'
folder_name = 'swin'

# config file 들고오기
cfg = Config.fromfile(f'./configs/{folder_name}/{model_name}.py')

In [5]:
PREFIX = '../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = [(666, 400), (999, 600), (1333, 800), (1666, 1000), (1999, 1200)]

cfg.data.test.pipeline[1].flip=True
cfg.data.workers_per_gpu = 4

cfg.seed=0
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{model_name}'

# cfg.model.bbox_head.num_classes = 11
# cfg.model.roi_head.bbox_head[0].num_classes = 11
# cfg.model.roi_head.bbox_head[1].num_classes = 11
# cfg.model.roi_head.bbox_head[2].num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_31.pth')

In [6]:
import pprint
pprint.pprint(cfg.data.test)

{'ann_file': '../input/data/test.json',
 'classes': ('UNKNOWN',
             'General trash',
             'Paper',
             'Paper pack',
             'Metal',
             'Glass',
             'Plastic',
             'Styrofoam',
             'Plastic bag',
             'Battery',
             'Clothing'),
 'img_prefix': '../input/data/',
 'pipeline': [{'type': 'LoadImageFromFile'},
              {'flip': True,
               'img_scale': [(666, 400),
                             (999, 600),
                             (1333, 800),
                             (1666, 1000),
                             (1999, 1200)],
               'transforms': [{'keep_ratio': True, 'type': 'Resize'},
                              {'type': 'RandomFlip'},
                              {'mean': [123.675, 116.28, 103.53],
                               'std': [58.395, 57.12, 57.375],
                               'to_rgb': True,
                               'type': 'Normalize'},
              

In [7]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [8]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader


In [9]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 0.7 task/s, elapsed: 1287s, ETA:     0s

In [11]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.29709038 151.27834 63.06493 255.81226 179....,batch_01_vt/0021.jpg
1,1 0.81775314 0.15429428 246.83818 306.55154 45...,batch_01_vt/0028.jpg
2,1 0.992358 70.80414 168.77917 374.22006 377.75...,batch_01_vt/0031.jpg
3,1 0.059699077 286.99963 151.0393 299.08276 159...,batch_01_vt/0032.jpg
4,0 0.07959624 38.109142 460.0278 98.12811 472.5...,batch_01_vt/0070.jpg
